In [41]:
import pandas as pd
from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
from langchain.prompts import ChatPromptTemplate

In [42]:
import os
os.environ["OPENAI_API_KEY"] = "sk-..."
os.environ["OPENAI_API_BASE"] = "https://api.groq.com/openai/v1"

In [43]:
df = pd.read_csv('analise_geral.csv')

In [44]:
# Opcional: também pode ler raça e deficiência
df_raca = pd.read_csv('analise_por_raca.csv')
df_deficiencia = pd.read_csv('analise_por_deficiencia.csv')

In [45]:
# 2. Preparar o resumo automático dos dados para enviar no prompt
# Simplificamos para não ultrapassar limite de tokens
resumo = []

In [46]:
resumo.append("### Análise Geral por Estado:")
for index, row in df.iterrows():
    resumo.append(f"Estado: {row['sigla_uf']}, Total Empregados: {row['total_empregados']}, Total Desempregados: {row['total_desempregados']}, Desligados por 1000 Matrículas: {row['desligados_por_1000_matriculas']:.2f}, Taxa Desemprego: {row['taxa_desemprego']:.2f}%, Relação Empregados/Desempregado: {row['empregados_por_desempregado']:.2f}")

resumo.append("\n### Análise por Raça:")
for index, row in df_raca.iterrows():
    resumo.append(f"Ano: {row['ano']}, Estado: {row['sigla_uf']}, Raça: {row['raca_cor']}, Empregados: {row['empregados_raca']}, Desempregados: {row['desempregados_raca']}")

resumo.append("\n### Análise por Deficiência:")
for index, row in df_deficiencia.iterrows():
    resumo.append(f"Ano: {row['ano']}, Estado: {row['sigla_uf']}, Deficiência: {row['indicador_portador_deficiencia']}, Empregados: {row['empregados_deficiencia']}, Desempregados: {row['desempregados_deficiencia']}")


In [47]:
# 3. Criar o template do prompt
prompt_template = ChatPromptTemplate.from_messages([
    SystemMessage(content="Você é um assistente de análise de dados acadêmico, que responde de forma impessoal e estruturada para relatórios de pós-graduação."),
    HumanMessage(content=f"""
Sou um analista de dados que está preparando um relatório para um trabalho de pós-graduação sobre o mercado de trabalho e educação no Brasil.

Baseando-se nestes dados:

{os.linesep.join(resumo[:100])}

Por favor, de forma estruturada e impessoal, forneça insights acadêmicos sobre:
- Quais são as regiões mais afetadas pelo desemprego.
- Quais grupos raciais e pessoas com deficiência enfrentam maiores dificuldades no emprego.
- Como o número de matrículas se relaciona com o desemprego.
- Destaque os padrões ou anomalias mais relevantes.
- Não use julgamentos, apenas análises técnicas baseadas nos dados fornecidos.

Formato da resposta:
1. Sumário Executivo
2. Regiões Críticas
3. Grupos Sociais Impactados
4. Conexões entre Educação e Trabalho
5. Conclusão
""")
])

In [48]:
# 4. Configurar o modelo do Groq via Langchain
chat = ChatOpenAI(
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

In [49]:
from langchain.schema import BaseOutputParser


class CustomMarkdownOutputParser(BaseOutputParser):
    def parse(self, text: str) -> str:
        """
        Lógica de parsing que simplesmente retorna o texto, sem alterações.
        """
        return text

    def invoke(self, input, config=None):
        """
        Método ajustado para retornar diretamente o texto recebido.
        :param input: Entrada, um texto que será processado.
        :param config: Argumento adicional (opcional) para configurações, mantido para compatibilidade.
        """
        # Apenas imprimimos para depuração, sem alterar o fluxo do método.
        print("Input recebido:", input)
        if config:
            print("Config recebido:", config)

        # Retorna o texto recebido, sem modificações.
        return input

    def __call__(self, input: str) -> str:
        """
        Permite que a instância seja chamada diretamente, retornando o texto sem alterações.
        """
        return self.invoke(input)



In [50]:
# 5. Configurar o parser de saída
parser = CustomMarkdownOutputParser()

In [51]:
# 6. Rodar a cadeia
chain = prompt_template | chat | parser

In [52]:
# 7. Executar e salvar
response = chain.invoke({})

Input recebido: content='## 1. Sumário Executivo\nEste relatório analisa a situação do mercado de trabalho e educação no Brasil, com foco em dados de emprego, desemprego e matrículas por estado e grupos raciais. Os resultados indicam que há disparidades significativas no desemprego entre diferentes estados e grupos raciais. Além disso, há uma correlação entre o número de matrículas e as taxas de desemprego.\n\n## 2. Regiões Críticas\n- **Taxas de Desemprego por Estado:** Os estados com as maiores taxas de desemprego são AL (49.64%), AC (49.76%) e MS (48.52%). Em contraste, RR apresenta a menor taxa de desemprego (45.74%).\n- **Total de Desempregados:** Os estados com o maior número de desempregados são SP (597,715), MG (244,788) e RJ (151,727). Esses estados concentram as maiores populações economicamente ativas e, portanto, também os maiores contingentes de desempregados.\n\n## 3. Grupos Sociais Impactados\n- **Grupos Raciais:** A análise por raça é limitada devido à presença de valor

In [57]:
response.content

'## 1. Sumário Executivo\nEste relatório analisa a situação do mercado de trabalho e educação no Brasil, com foco em dados de emprego, desemprego e matrículas por estado e grupos raciais. Os resultados indicam que há disparidades significativas no desemprego entre diferentes estados e grupos raciais. Além disso, há uma correlação entre o número de matrículas e as taxas de desemprego.\n\n## 2. Regiões Críticas\n- **Taxas de Desemprego por Estado:** Os estados com as maiores taxas de desemprego são AL (49.64%), AC (49.76%) e MS (48.52%). Em contraste, RR apresenta a menor taxa de desemprego (45.74%).\n- **Total de Desempregados:** Os estados com o maior número de desempregados são SP (597,715), MG (244,788) e RJ (151,727). Esses estados concentram as maiores populações economicamente ativas e, portanto, também os maiores contingentes de desempregados.\n\n## 3. Grupos Sociais Impactados\n- **Grupos Raciais:** A análise por raça é limitada devido à presença de valores não numéricos (nan) e

In [58]:
resultado = parser(response.content)

Input recebido: ## 1. Sumário Executivo
Este relatório analisa a situação do mercado de trabalho e educação no Brasil, com foco em dados de emprego, desemprego e matrículas por estado e grupos raciais. Os resultados indicam que há disparidades significativas no desemprego entre diferentes estados e grupos raciais. Além disso, há uma correlação entre o número de matrículas e as taxas de desemprego.

## 2. Regiões Críticas
- **Taxas de Desemprego por Estado:** Os estados com as maiores taxas de desemprego são AL (49.64%), AC (49.76%) e MS (48.52%). Em contraste, RR apresenta a menor taxa de desemprego (45.74%).
- **Total de Desempregados:** Os estados com o maior número de desempregados são SP (597,715), MG (244,788) e RJ (151,727). Esses estados concentram as maiores populações economicamente ativas e, portanto, também os maiores contingentes de desempregados.

## 3. Grupos Sociais Impactados
- **Grupos Raciais:** A análise por raça é limitada devido à presença de valores não numéricos 

In [59]:
with open('insights_gerados_llm.txt', 'w', encoding='utf-8') as f:
    f.write(resultado)

print("✅ Insights gerados com Langchain e GroqCloud e salvos em 'insights_gerados_llm.txt'.")

✅ Insights gerados com Langchain e GroqCloud e salvos em 'insights_gerados_llm.txt'.
